In [1]:
%load_ext lab_black
import pandas as pd
from IPython.display import JSON
import warnings
import ast

warnings.filterwarnings("ignore")

In [140]:
df = pd.read_csv("world_cup_2022_catar.csv")
df.shape

(64, 25)

In [7]:
df.head(1)

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,...,duels_won_home,duels_won_away,prediction_team_home_win,prediction_draw,prediction_team_away_win,prediction_quantity,location,events_list,lineup_home,lineup_away
0,Final,18/12/2022,True,4,2,Argentina,France,3,3,54%,...,40%,60%,64%,3%,33%,1002332,Lusail Iconic Stadium,"[{'event_team': 'home', 'event_time': "" 23' "",...","[{'player_name': ' Di María ', 'player_number'...","[{'player_name': ' Giroud ', 'player_number': ..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   stage                     64 non-null     object
 1   date                      64 non-null     object
 2   pens                      64 non-null     bool  
 3   pens_home_score           64 non-null     object
 4   pens_away_score           64 non-null     object
 5   team_name_home            64 non-null     object
 6   team_name_away            64 non-null     object
 7   team_home_score           64 non-null     int64 
 8   team_away_score           64 non-null     int64 
 9   possession_home           64 non-null     object
 10  possession_away           64 non-null     object
 11  total_shots_home          64 non-null     int64 
 12  total_shots_away          64 non-null     int64 
 13  shots_on_target_home      64 non-null     int64 
 14  shots_on_target_away      64

In [5]:
df.describe()

,team_home_score,team_away_score,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,prediction_quantity
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,6.400000e+01
mean,1.578125,1.109375,11.296875,11.453125,4.312500,3.765625,4.036118e+05
std,1.551289,1.055856,5.088673,5.866043,2.617857,2.729860,2.001993e+05
min,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.304210e+05
25%,0.000000,0.000000,8.000000,7.750000,3.000000,2.000000,2.321788e+05
50%,1.000000,1.000000,10.500000,10.500000,4.000000,3.000000,3.639255e+05
75%,2.000000,2.000000,14.000000,14.000000,6.000000,5.000000,5.351200e+05
max,7.000000,4.000000,26.000000,32.000000,10.000000,12.000000,1.002332e+06


In [8]:
df.columns

Index(['stage', 'date', 'pens', 'pens_home_score', 'pens_away_score',
       'team_name_home', 'team_name_away', 'team_home_score',
       'team_away_score', 'possession_home', 'possession_away',
       'total_shots_home', 'total_shots_away', 'shots_on_target_home',
       'shots_on_target_away', 'duels_won_home', 'duels_won_away',
       'prediction_team_home_win', 'prediction_draw',
       'prediction_team_away_win', 'prediction_quantity', 'location',
       'events_list', 'lineup_home', 'lineup_away'],
      dtype='object')

### Manipulating events_list, lineup_home and lineup_away columns into new columns

In [141]:
df["lineup_home"] = df["lineup_home"].str.replace("'", '"')
df["lineup_home"] = df["lineup_home"].apply(json.loads)

# transformar as colunas
def extract_player_data(lineup):
    if not lineup or not isinstance(lineup, list):
        return None, None
    player_names = [
        player["player_name"] for player in lineup if isinstance(player, dict)
    ]
    player_numbers = [
        player["player_number"] for player in lineup if isinstance(player, dict)
    ]
    if len(player_names) != len(player_numbers):
        print(lineup)
        return None, None
    return (player_names, player_numbers)


df["player_names_home"] = df.apply(
    lambda row: extract_player_data(row["lineup_home"])[0], axis=1
)
df["player_numbers_home"] = df.apply(
    lambda row: extract_player_data(row["lineup_home"])[1], axis=1
)

# retirar o espaçamento entre os valores
df["player_names_home"] = df["player_names_home"].apply(
    lambda x: list(map(str.strip, x))
)
df["player_numbers_home"] = df["player_numbers_home"].apply(
    lambda x: list(map(str.strip, x))
)

In [142]:
df["lineup_away"] = df["lineup_away"].str.replace("'", '"')
df["lineup_away"] = df["lineup_away"].apply(json.loads)

# transformar as colunas
def extract_player_data(lineup):
    if not lineup or not isinstance(lineup, list):
        return None, None
    player_names = [
        player["player_name"] for player in lineup if isinstance(player, dict)
    ]
    player_numbers = [
        player["player_number"] for player in lineup if isinstance(player, dict)
    ]
    if len(player_names) != len(player_numbers):
        print(lineup)
        return None, None
    return (player_names, player_numbers)


df["player_names_away"] = df.apply(
    lambda row: extract_player_data(row["lineup_away"])[0], axis=1
)
df["player_numbers_away"] = df.apply(
    lambda row: extract_player_data(row["lineup_away"])[1], axis=1
)

# retirar o espaçamento entre os valores
df["player_names_away"] = df["player_names_away"].apply(
    lambda x: list(map(str.strip, x))
)
df["player_numbers_away"] = df["player_numbers_away"].apply(
    lambda x: list(map(str.strip, x))
)

In [143]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,...,prediction_team_away_win,prediction_quantity,location,events_list,lineup_home,lineup_away,player_names_home,player_numbers_home,player_names_away,player_numbers_away
0,Final,18/12/2022,True,4,2,Argentina,France,3,3,54%,...,33%,1002332,Lusail Iconic Stadium,"[{'event_team': 'home', 'event_time': "" 23' "",...","[{'player_name': ' Di María ', 'player_number'...","[{'player_name': ' Giroud ', 'player_number': ...","[Di María, Álvarez, Messi, Mac Allister, Ferná...","[11, 9, 10, 20, 24, 7, 3, 19, 13, 26, 23]","[Giroud, Mbappé, Griezmann, Dembélé, Rabiot, T...","[9, 10, 7, 11, 14, 8, 22, 18, 4, 5, 1]"
1,Match for third place,17/12/2022,False,False,False,Croatia,Morocco,2,1,51%,...,54%,367156,Khalifa International Stadium,"[{'event_team': 'home', 'event_time': "" 7' "", ...","[{'player_name': ' Livaja ', 'player_number': ...","[{'player_name': ' Boufal ', 'player_number': ...","[Livaja, Oršić, Kramarić, Majer, Kovačić, Modr...","[14, 18, 9, 7, 8, 10, 4, 20, 24, 2, 1]","[Boufal, En-Nesyri, Ziyech, Sabiri, Amrabat, E...","[17, 19, 7, 11, 4, 23, 25, 20, 18, 2, 1]"
2,Semi-finals,14/12/2022,False,False,False,France,Morocco,2,0,38%,...,39%,720768,Al Bayt Stadium,"[{'event_team': 'home', 'event_time': "" 5' "", ...","[{'player_name': ' Giroud ', 'player_number': ...","[{'player_name': ' En-Nesyri ', 'player_number...","[Giroud, Mbappé, Griezmann, Dembélé, Fofana, T...","[9, 10, 7, 11, 13, 8, 22, 24, 4, 5, 1]","[En-Nesyri, Boufal, Amrabat, Ounahi, Ziyech, M...","[19, 17, 4, 8, 7, 3, 18, 6, 20, 2, 1]"
3,Semi-finals,13/12/2022,False,False,False,Argentina,Croatia,3,0,39%,...,25%,792326,Lusail Iconic Stadium,"[{'event_team': 'away', 'event_time': "" 32' "",...","[{'player_name': ' Álvarez ', 'player_number':...","[{'player_name': ' Perišić ', 'player_number':...","[Álvarez, Messi, Mac Allister, Fernández, Pare...","[9, 10, 20, 24, 5, 7, 3, 19, 13, 26, 23]","[Perišić, Kramarić, Pašalić, Kovačić, Brozović...","[4, 9, 15, 8, 11, 10, 19, 20, 6, 22, 1]"
4,Quarter-finals,10/12/2022,False,False,False,England,France,1,2,58%,...,55%,672577,Al Bayt Stadium,"[{'event_team': 'away', 'event_time': "" 17' "",...","[{'player_name': ' Foden ', 'player_number': '...","[{'player_name': ' Giroud ', 'player_number': ...","[Foden, Kane, Saka, Bellingham, Rice, Henderso...","[20, 9, 17, 22, 4, 8, 3, 6, 5, 2, 1]","[Giroud, Mbappé, Griezmann, Dembélé, Rabiot, T...","[9, 10, 7, 11, 14, 8, 22, 18, 4, 5, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Group stage: Matchday 1,22/11/2022,False,False,False,Argentina,Saudi Arabia,1,2,70%,...,4%,657383,Lusail Iconic Stadium,"[{'event_team': 'home', 'event_time': "" 10' "",...","[{'player_name': ' Martínez ', 'player_number'...","[{'player_name': ' Al-Shehri ', 'player_number...","[Martínez, Messi, Gómez, Paredes, De Paul, Di ...","[22, 10, 17, 5, 7, 11, 3, 19, 13, 26, 23]","[Al-Shehri, Al-Dawsari, Kanno, Al-Faraj, Al-Bu...","[11, 10, 23, 7, 9, 8, 13, 5, 17, 12, 21]"
60,Group stage: Matchday 1,21/11/2022,False,False,False,USA,Wales,1,1,59%,...,40%,361145,Ahmad bin Ali Stadium,"[{'event_team': 'home', 'event_time': "" 11' "",...","[{'player_name': ' Pulisic ', 'player_number':...","[{'player_name': ' James ', 'player_number': '...","[Pulisic, Sargent, Weah, Musah, Adams, McKenni...","[10, 24, 21, 6, 4, 8, 5, 13, 3, 2, 1]","[James, Bale, Williams, Wilson, Ampadu, Ramsey...","[20, 11, 3, 8, 15, 10, 14, 4, 6, 5, 1]"
61,Group stage: Matchday 1,21/11/2022,False,False,False,Senegal,Netherlands,0,2,46%,...,71%,457977,Al Thumama Stadium,"[{'event_team': 'away', 'event_time': "" 56' "",...","[{'player_name': ' Dia ', 'player_number': ' 9...","[{'player_name': ' Bergwijn ', 'player_number'...","[Dia, Sarr, Gueye, Diatta, Mendy, Kouyaté, Dia...","[9, 18, 5, 15, 6, 8, 22, 4, 3, 21, 16]","[Bergwijn, Janssen, Gakpo, Blind, De Jong, Ber...","[7, 18, 8, 17, 21, 11, 22,

In [149]:
list(df.columns.values.tolist())

['stage',
 'date',
 'pens',
 'pens_home_score',
 'pens_away_score',
 'team_name_home',
 'team_name_away',
 'team_home_score',
 'team_away_score',
 'possession_home',
 'possession_away',
 'total_shots_home',
 'total_shots_away',
 'shots_on_target_home',
 'shots_on_target_away',
 'duels_won_home',
 'duels_won_away',
 'prediction_team_home_win',
 'prediction_draw',
 'prediction_team_away_win',
 'prediction_quantity',
 'location',
 'events_list',
 'lineup_home',
 'lineup_away',
 'player_names_home',
 'player_numbers_home',
 'player_names_away',
 'player_numbers_away',
 'event_team',
 'event_time',
 'event_type',
 'action_player_1',
 'action_player_2']

In [148]:
df.events_list[0]

'[{\'event_team\': \'home\', \'event_time\': " 23\' ", \'event_type\': \'Penalty\', \'action_player_1\': \' Lionel Messi \', \'action_player_2\': \' Penalty \'}, {\'event_team\': \'home\', \'event_time\': " 36\' ", \'event_type\': \'Goal\', \'action_player_1\': \' Ángel Di María \', \'action_player_2\': \' Alexis Mac Allister \'}, {\'event_team\': \'away\', \'event_time\': " 41\' ", \'event_type\': \'Substitution\', \'action_player_1\': \' Randal Kolo Muani \', \'action_player_2\': \' Ousmane Dembélé \'}, {\'event_team\': \'away\', \'event_time\': " 41\' ", \'event_type\': \'Substitution\', \'action_player_1\': \' Marcus Thuram \', \'action_player_2\': \' Olivier Giroud \'}, {\'event_team\': \'home\', \'event_time\': " 52\' ", \'event_type\': \'Yellow card\', \'action_player_1\': \' Enzo Fernández \'}, {\'event_team\': \'away\', \'event_time\': " 55\' ", \'event_type\': \'Yellow card\', \'action_player_1\': \' Adrien Rabiot \'}, {\'event_team\': \'home\', \'event_time\': " 64\' ", \'ev

In [171]:
# cria uma nova tabela vazia
df_events_list = pd.DataFrame(
    columns=[
        "team",
        "event_team",
        "event_time",
        "event_type",
        "action_player_1",
        "action_player_2",
    ]
)

for i, row in df.iterrows():
    events_list = ast.literal_eval(row["events_list"])
    for event in events_list:
        if event.get("event_team") == "home":
            team = row.team_name_home
        else:
            team = row.team_name_away
        df_events_list = df_events_list.append(
            {
                "team": team,
                "event_team": event.get("event_team"),
                "event_time": event.get("event_time"),
                "event_type": event.get("event_type"),
                "action_player_1": event.get("action_player_1"),
                "action_player_2": event.get("action_player_2"),
            },
            ignore_index=True,
        )

In [172]:
df_events_list

,team,event_team,event_time,event_type,action_player_1,action_player_2
0,Argentina,home,23',Penalty,Lionel Messi,Penalty
1,Argentina,home,36',Goal,Ángel Di María,Alexis Mac Allister
2,France,away,41',Substitution,Randal Kolo Muani,Ousmane Dembélé
3,France,away,41',Substitution,Marcus Thuram,Olivier Giroud
4,Argentina,home,52',Yellow card,Enzo Fernández,None
...,...,...,...,...,...,...
1038,Qatar,home,72',Substitution,Mohammed Muntari,Almoez Ali
1039,Ecuador,away,77',Substitution,José Cifuentes,Enner Valencia
1040,Qatar,home,78',Yellow card,Akram Afif,None
1041,Ecuador,away,90',Substitution,Kevin Rodríguez,Michael Estrada
